In [140]:
import json
import pandas as pd
import networkx as nx
import folium
from geopy.distance import geodesic
from scipy.spatial import KDTree
import geopandas as gpd

In [141]:
# 데이터 로드
with open('../../../../data/04_final_data/final_edges.geojson', encoding='utf-8') as f:
    edges_geo = json.load(f)
nodes_df = pd.read_csv('../../../../data/04_final_data/final_nodes.csv')
tree_char_df = pd.read_csv('../../../../data/02_intermediate/tree-characteristics.csv')

In [142]:
# 계절별 적합 나무 dict 생성
def get_seasonal_trees(season):
    return set(tree_char_df[tree_char_df[season] == 1]['수목명'])

# 노드 위치 dict
node_pos = {row['osmid']: (row['lat'], row['lon']) for _, row in nodes_df.iterrows()}

In [143]:
# 그래프 생성
G = nx.Graph()
for feature in edges_geo['features']:
    props = feature['properties']
    u, v = props['u'], props['v']
    length = props.get('length', 1)
    type_keys = ['park', 'mountain', 'river', 'tree-line', 'road']
    edge_type = next((k for k in type_keys if props.get(k, 0) == 1), 'road')
    tree_names = props.get('tree', '').split()
    G.add_edge(u, v, length=length, type=edge_type, tree=tree_names)

In [144]:
def edge_weight(u, v, season):
    data = G[u][v]
    # 도로 유형별 가중치 (낮을수록 선호)
    type_weight = {
        'park': 0.5,
        'river': 0.5,
        'mountain': 0.5,
        'tree_line': 0.5,
        'road': 2
    }
    w = type_weight.get(data['type'], 1.5)
    # 계절 적합 나무 포함시 가중치 추가 감소
    seasonal_trees = get_seasonal_trees(season)
    if any(t in seasonal_trees for t in data['tree']):
        w *= 0.7
    return w * data['length']

In [145]:
# 경로 추천 알고리즘
def recommend_circular_path(G, node_pos, start_lat, start_lon, season, walk_km, tolerance=0.2):
    # 시작점과 가장 가까운 그래프 상의 노드 찾기
    node_coords = list(node_pos.values())
    node_osmids = list(node_pos.keys())
    kdtree = KDTree(node_coords)
    _, start_node_idx = kdtree.query([start_lat, start_lon])
    start_osmid = node_osmids[start_node_idx]

    # 시작점 주변 서브그래프 생성
    subG = filter_graph_by_bbox_and_type(G, node_pos, start_lat, start_lon, walk_km)
    if not subG.has_node(start_osmid):
        print("오류: 시작점 주변에 탐색할 경로가 없습니다.")
        return None
        
    # 모든 엣지에 대해 동적 가중치 계산 및 적용
    for u, v in subG.edges():
        subG[u][v]['weight'] = edge_weight(u, v, season)

    # '매력적인' 목표 지점(POI) 찾기 (가중치 기반)
    edge_attractiveness = []
    seasonal_trees = get_seasonal_trees(season)

    for u, v, data in subG.edges(data=True):
        # --- [수정된 부분 시작] ---
        # 아래 로직 전체를 for 루프 안으로 이동
        type_keys = ['park', 'river', 'mountain', 'tree_line', 'road']
        edge_type = next((key for key in type_keys if data.get(key) == 1), 'road')

        type_multiplier = {
            'park': 0.2, 'river': 0.2, 'mountain': 0.2, 
            'tree_line': 1.3, 'road': 2.0
        }.get(edge_type)
        
        # 안전장치: edge_type이 딕셔너리에 없는 경우 기본값 사용
        if type_multiplier is None:
            type_multiplier = 1.5

        tree_multiplier = 0.9 if any(t in seasonal_trees for t in data.get('tree', [])) else 1.0
        attractiveness_score = 1 / (type_multiplier * tree_multiplier)
        edge_attractiveness.append(((u, v), attractiveness_score))
        # --- [수정된 부분 끝] ---

    # 매력도 점수가 높은 상위 10%의 엣지를 POI로 선정
    edge_attractiveness.sort(key=lambda x: x[1], reverse=True)
    top_10_percent_index = int(len(edge_attractiveness) * 0.1)
    
    if top_10_percent_index > 0:
        top_edges = [edge for edge, score in edge_attractiveness[:top_10_percent_index]]
        pois = {n for u, v in top_edges for n in (u, v)}
    else:
        pois = set()

    if not pois:
        print("알림: 주변에 특별히 추천할 만한 경로가 없어 일반 경로를 탐색합니다.")
        pois = {n for u, v, data in subG.edges(data=True) if data.get('type') != 'road' for n in (u,v)}

    # 다익스트라 경로 계산
    try:
        path_weights = nx.single_source_dijkstra_path_length(subG, start_osmid, weight='weight')
        path_lengths = nx.single_source_dijkstra_path_length(subG, start_osmid, weight='length')
    except nx.NetworkXNoPath:
        print(f"오류: 시작점 {start_osmid}이(가) 주변 경로와 연결되어 있지 않습니다.")
        return None

    # POI까지의 왕복 경로 후보군 필터링
    candidates = []
    walk_m = walk_km * 1000
    for poi in pois:
        if poi in path_lengths:
            round_trip_m = path_lengths[poi] * 2
            if abs(round_trip_m - walk_m) <= walk_m * tolerance:
                candidates.append((poi, round_trip_m))
    
    if not candidates:
        print(f"{walk_km}km 근방의 추천 경로를 찾지 못했습니다.")
        return None
        
    # 최적 경로 선택 및 구성
    best_poi, best_length = min(candidates, key=lambda x: abs(x[1] - walk_m))
    path_to_poi = nx.shortest_path(subG, start_osmid, best_poi, weight='weight')
    path_from_poi = nx.shortest_path(subG, best_poi, start_osmid, weight='weight')
    final_path = path_to_poi + path_from_poi[1:]
    
    return final_path

In [139]:


# --- 사용자 입력 변수 ---
start_lat, start_lon = 37.567893, 127.051643 
walk_km = 3  # 3km 산책
season = '여름'  # 계절: 봄

# --- 경로 추천 함수 호출 ---
path_nodes = recommend_circular_path(G, node_pos, start_lat, start_lon, season, walk_km)

# --- 결과 시각화 (기존 코드와 연결) ---
if path_nodes:
    # 경로 길이 계산
    total_length = sum(G.edges[path_nodes[i], path_nodes[i+1]]['length'] for i in range(len(path_nodes)-1))
    print(f'추천 경로 노드 수: {len(path_nodes)}, 실제 경로 길이: {total_length:.1f} m ({total_length/1000:.2f} km)')
    
    # 지도 생성
    m = folium.Map(location=[start_lat, start_lon], zoom_start=15)
    folium.Marker(location=[start_lat, start_lon], popup="Start/End", icon=folium.Icon(color='red')).add_to(m)

    # 경로 PolyLine 생성
    path_coords = [node_pos[node] for node in path_nodes]
    folium.PolyLine(
        locations=path_coords,
        color='blue',
        weight=5,
        opacity=0.8
    ).add_to(m)
    
    # 지도 저장
    m.save('recommended_walk_path.html')
    print("'recommended_walk_path.html' 파일이 저장되었습니다.")
else:
    print('경로를 찾지 못하여 지도를 생성하지 않았습니다.')

추천 경로 노드 수: 101, 실제 경로 길이: 3218.6 m (3.22 km)
'recommended_walk_path.html' 파일이 저장되었습니다.


In [ ]:
'''
# 지도 생성 및 경로 정보 출력

if path_nodes is None or len(path_nodes) < 2:
    print('시각화할 경로가 없습니다. 조건을 다시 조정해 주세요.')
else:
    # 경로 길이 계산
    total_length = sum(G[path_nodes[i]][path_nodes[i+1]]['length'] for i in range(len(path_nodes)-1))
    print(f'경로 노드 수: {len(path_nodes)}, 경로 길이: {total_length:.1f} m')
    m = folium.Map(location=[start_lat, start_lon], zoom_start=15)

    # 경로 노드 표시
    for osmid in path_nodes:
        lat, lon = node_pos[osmid]
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            color='gray',
            fill=True,
            fill_opacity=0.8
        ).add_to(m)

    # 경로 엣지 표시
    for i in range(len(path_nodes)-1):
        u, v = path_nodes[i], path_nodes[i+1]
        edge = G[u][v]
        color_map = {
            'park': 'yellow',
            'tree_line': 'green',
            'river': 'blue',
            'mountain': 'pink',
            'road': 'black'
        }
        color = color_map.get(edge['type'], 'black')
        folium.PolyLine(
            locations=[node_pos[u], node_pos[v]],
            color=color,
            weight=5,
            opacity=0.8
        ).add_to(m)

    # 지도 저장
    m.save('recommended_walk_path.html')
'''

경로 노드 수: 11, 경로 길이: 611.7 m
